In [1]:
!pip install wandb pytorch-ssim


  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-ssim: filename=pytorch_ssim-0.1-py3-none-any.whl size=2006 sha256=69ec432f75f3f84bd97046b1c3de01db5916ab7981ddb1ab22f83becc5b72a95
  Stored in directory: /root/.cache/pip/wheels/54/a0/11/99f86224e71729ed9ef0c4ffe1b795807ad5f44bde19bc66f9
Successfully built pytorch-ssim


In [2]:
!pip install kagglehub
!pip install kaggle


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rajat95gupta/smartphone-image-denoising-dataset")

print("Path to dataset files:", path)

100%|██████████| 6.16G/6.16G [00:57<00:00, 114MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rajat95gupta/smartphone-image-denoising-dataset/versions/1


In [4]:
import os

dataset_root = "/root/.cache/kagglehub/datasets/rajat95gupta/smartphone-image-denoising-dataset/versions/1"

# List all files and subfolders
for root, dirs, files in os.walk(dataset_root):
    print("DIR:", root)
    for d in dirs:
        print("  ├─ Subdir:", d)
    for f in files[:5]:  # just show a few files
        print("  └─ File:", f)
    print()


DIR: /root/.cache/kagglehub/datasets/rajat95gupta/smartphone-image-denoising-dataset/versions/1
  ├─ Subdir: SIDD_Small_sRGB_Only

DIR: /root/.cache/kagglehub/datasets/rajat95gupta/smartphone-image-denoising-dataset/versions/1/SIDD_Small_sRGB_Only
  ├─ Subdir: Data
  └─ File: _ReadMe.txt
  └─ File: Scene_Instances.txt

DIR: /root/.cache/kagglehub/datasets/rajat95gupta/smartphone-image-denoising-dataset/versions/1/SIDD_Small_sRGB_Only/Data
  ├─ Subdir: 0042_002_IP_01600_03100_5500_N
  ├─ Subdir: 0066_003_GP_00100_00200_3200_L
  ├─ Subdir: 0097_005_N6_03200_02000_3200_L
  ├─ Subdir: 0118_006_N6_00100_00025_3200_L
  ├─ Subdir: 0114_005_IP_00100_00200_5500_N
  ├─ Subdir: 0040_002_IP_00800_02000_5500_L
  ├─ Subdir: 0115_005_IP_00400_00750_5500_N
  ├─ Subdir: 0013_001_S6_03200_01250_3200_L
  ├─ Subdir: 0048_002_N6_00100_00100_5500_L
  ├─ Subdir: 0034_002_GP_00100_00160_3200_N
  ├─ Subdir: 0159_007_IP_00100_00100_3200_L
  ├─ Subdir: 0191_008_IP_01600_01600_3200_N
  ├─ Subdir: 0185_008_IP_0040

In [5]:
import os
import shutil

source_data_dir = "/root/.cache/kagglehub/datasets/rajat95gupta/smartphone-image-denoising-dataset/versions/1/SIDD_Small_sRGB_Only/Data"
output_noisy_dir = "/content/SIDD/input"
output_clean_dir = "/content/SIDD/target"

# Create target folders
os.makedirs(output_noisy_dir, exist_ok=True)
os.makedirs(output_clean_dir, exist_ok=True)

# Initialize image counter
copied = 0

# Walk through each scene directory
for scene_dir in sorted(os.listdir(source_data_dir)):
    scene_path = os.path.join(source_data_dir, scene_dir)
    if not os.path.isdir(scene_path):
        continue

    # Get list of image files
    files = sorted(os.listdir(scene_path))

    # Group noisy and clean images
    noisy_files = [f for f in files if "NOISY" in f.upper()]
    clean_files = [f for f in files if "GT" in f.upper()]

    # Sort to ensure alignment
    noisy_files.sort()
    clean_files.sort()

    # Copy files
    for noisy_file, clean_file in zip(noisy_files, clean_files):
        shutil.copy(os.path.join(scene_path, noisy_file), os.path.join(output_noisy_dir, f"{copied:05d}.png"))
        shutil.copy(os.path.join(scene_path, clean_file), os.path.join(output_clean_dir, f"{copied:05d}.png"))
        copied += 1

print(f"Copied {copied} noisy/clean image pairs.")


Copied 160 noisy/clean image pairs.


In [9]:
# denoise_prune_export.py
import os
import glob
from math import exp
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms as T

import torch.nn.utils.prune as prune
import wandb  # Import wandb

# -----------------------
# Dataset (same as you)
# -----------------------
class SIDD_DenoiseDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, transform=None):
        self.noisy_images = sorted(glob.glob(os.path.join(noisy_dir, "*.png")))
        self.clean_images = sorted(glob.glob(os.path.join(clean_dir, "*.png")))
        self.transform = transform
        assert len(self.noisy_images) == len(self.clean_images), "Mismatch in noisy and clean image counts"

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        noisy = Image.open(self.noisy_images[idx]).convert("RGB")
        clean = Image.open(self.clean_images[idx]).convert("RGB")
        if self.transform:
            noisy = self.transform(noisy)
            clean = self.transform(clean)
        return noisy.clamp(0., 1.), clean

# -----------------------
# Lightweight building blocks
# -----------------------
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, max(in_planes // ratio, 4), 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(max(in_planes // ratio, 4), in_planes, 1)
        )
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        return x * self.sigmoid(self.fc(self.avg_pool(x)))

class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        max_pool = torch.max(x, dim=1, keepdim=True)[0]
        avg_pool = torch.mean(x, dim=1, keepdim=True)
        pooled = torch.cat([avg_pool, max_pool], dim=1)
        return self.sigmoid(self.conv(pooled))

class CBAM(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.ca = ChannelAttention(in_channels)
        self.sa = SpatialAttention()
    def forward(self, x):
        return self.ca(x) * self.sa(x)

def conv_block(in_ch, out_ch, separable=False):
    # Use regular convs but keep channels small for edge
    if separable:
        # depthwise separable conv (lightweight)
        return nn.Sequential(
            nn.Conv2d(in_ch, in_ch, kernel_size=3, padding=1, groups=in_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            CBAM(out_ch)
        )
    else:
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            CBAM(out_ch)
        )

class UNetStage(nn.Module):
    def __init__(self, in_ch=3, base_ch=32, separable=False):
        super().__init__()
        self.enc1 = conv_block(in_ch, base_ch, separable=separable)
        self.enc2 = conv_block(base_ch, base_ch*2, separable=separable)
        self.enc3 = conv_block(base_ch*2, base_ch*4, separable=separable)
        self.pool = nn.MaxPool2d(2)
        self.dec2 = conv_block(base_ch*4+base_ch*2, base_ch*2, separable=separable)
        self.dec1 = conv_block(base_ch*2+base_ch, base_ch, separable=separable)
        self.final = nn.Conv2d(base_ch, 3, kernel_size=1)
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        d2 = self.dec2(torch.cat([F.interpolate(e3, size=e2.shape[-2:], mode="bilinear", align_corners=False), e2], dim=1))
        d1 = self.dec1(torch.cat([F.interpolate(d2, size=e1.shape[-2:], mode="bilinear", align_corners=False), e1], dim=1))
        return self.final(d1)

class RefinementModule(nn.Module):
    """
    Lightweight refinement module that enhances details.
    This does NOT change spatial resolution (no upscaling). For an SR-like effect,
    it uses residual convs and channel attention to boost high-frequency details.
    """
    def __init__(self, channels=3, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(channels, hidden, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, hidden, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, kernel_size=3, padding=1)
        )
        self.ca = ChannelAttention(channels)
    def forward(self, x):
        res = self.net(x)
        out = x + res
        out = self.ca(out)
        return out

# -----------------------
# 2-stage MultiStage Restoration Net (lightweight)
# -----------------------
class MultiStageRestorationNet(nn.Module):
    def __init__(self, in_ch=3, base_ch=32, separable=False):
        super().__init__()
        self.stage1 = UNetStage(in_ch, base_ch=base_ch, separable=separable)
        self.stage2 = UNetStage(in_ch, base_ch=base_ch, separable=separable)
        self.refine = RefinementModule(channels=3, hidden=64)

    def forward(self, x):
        s1 = self.stage1(x)
        s2 = self.stage2(x - s1) + s1
        out = self.refine(s2)
        # clamp to valid range
        return out.clamp(0., 1.)

# -----------------------
# SSIM/PSNR utilities (kept similar to yours, minor fix to PSNR)
# -----------------------
def create_window(window_size, channel):
    def gaussian(window_size, sigma):
        g = torch.Tensor([exp(-(x-window_size//2)**2/(2*sigma**2)) for x in range(window_size)])
        return g/g.sum()
    _1D = gaussian(window_size, 1.5).unsqueeze(1)
    _2D = _1D.mul(_1D.t()).unsqueeze(0).unsqueeze(0)
    return _2D.expand(channel, 1, window_size, window_size).contiguous()

def _ssim(img1, img2, window, window_size, channel, size_average=True):
    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)
    mu1_sq, mu2_sq, mu1_mu2 = mu1**2, mu2**2, mu1*mu2
    sigma1_sq = F.conv2d(img1*img1, window, padding=window_size//2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding=window_size//2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding=window_size//2, groups=channel) - mu1_mu2
    C1, C2 = 0.01**2, 0.03**2
    ssim_map = ((2*mu1_mu2+C1)*(2*sigma12+C2))/((mu1_sq+mu2_sq+C1+1e-8)*(sigma1_sq+sigma2_sq+C2+1e-8))
    return ssim_map.mean() if size_average else ssim_map.mean([2,3]).mean(1)

def ssim_loss(output, target, window_size=11):
    (_, c, _, _) = output.size()
    window = create_window(window_size, c).to(output.device)
    return 1 - _ssim(output, target, window, window_size, c, True)

def psnr(output, target):
    mse = F.mse_loss(output, target, reduction="mean")
    # make sure numerical stability; if mse==0 -> very large PSNR
    if mse.item() == 0:
        return torch.tensor(100.0, device=output.device)
    return 10.0 * torch.log10(1.0 / mse)

# -----------------------
# Pruning utilities
# -----------------------
def apply_global_unstructured_pruning(model, amount=0.5):
    """
    Globally prune all Conv2d weight tensors by L1 magnitude.
    amount: fraction (0.0 - 1.0) of connections to prune globally.
    """
    parameters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            parameters_to_prune.append((module, "weight"))

    print(f"Applying global unstructured L1 pruning to {len(parameters_to_prune)} conv layers with amount={amount}")
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=amount,
    )

def remove_pruning_reparametrization(model):
    """
    Convert pruned parameters (which are currently masked) into permanent sparse params by removing
    the pruning reparametrization. After this, the model weights reflect pruning and masks are gone.
    """
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            try:
                prune.remove(module, "weight")
            except Exception:
                pass  # some modules may not be pruned

# -----------------------
# Training / Fine-tune loops
# -----------------------
def train_one_epoch(model, dataloader, opt, device, epoch):
    model.train()
    total_loss = 0.0
    total_psnr = 0.0
    total_l1 = 0.0
    total_ssim = 0.0
    total_mse = 0.0

    for noisy, clean in tqdm(dataloader, desc="Train"):
        noisy = noisy.to(device)
        clean = clean.to(device)
        out = model(noisy)
        l1 = F.l1_loss(out, clean)
        ssim_l = ssim_loss(out, clean)
        # small MSE term to stabilize refinement
        mse = F.mse_loss(out, clean)
        loss = l1 + 0.15 * ssim_l + 0.01 * mse

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss += loss.item()
        total_psnr += psnr(out, clean).item()
        total_l1 += l1.item()
        total_ssim += ssim_l.item()
        total_mse += mse.item()

    avg_loss = total_loss / len(dataloader)
    avg_psnr = total_psnr / len(dataloader)
    avg_l1 = total_l1 / len(dataloader)
    avg_ssim = total_ssim / len(dataloader)
    avg_mse = total_mse / len(dataloader)

    return avg_loss, avg_psnr, avg_l1, avg_ssim, avg_mse

def validate(model, dataloader, device, epoch):
    model.eval()
    total_loss = 0.0
    total_psnr = 0.0
    total_l1 = 0.0
    total_ssim = 0.0
    total_mse = 0.0

    with torch.no_grad():
        for noisy, clean in tqdm(dataloader, desc="Val"):
            noisy = noisy.to(device)
            clean = clean.to(device)
            out = model(noisy)
            l1 = F.l1_loss(out, clean)
            ssim_l = ssim_loss(out, clean)
            mse = F.mse_loss(out, clean)
            loss = l1 + 0.15 * ssim_l + 0.01 * mse

            total_loss += loss.item()
            total_psnr += psnr(out, clean).item()
            total_l1 += l1.item()
            total_ssim += ssim_l.item()
            total_mse += mse.item()

    avg_loss = total_loss / len(dataloader)
    avg_psnr = total_psnr / len(dataloader)
    avg_l1 = total_l1 / len(dataloader)
    avg_ssim = total_ssim / len(dataloader)
    avg_mse = total_mse / len(dataloader)

    return avg_loss, avg_psnr, avg_l1, avg_ssim, avg_mse

# -----------------------
# Main training + pruning + fine-tune + export
# -----------------------
def main():
    # ---- config ----
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    NOISY_DIR = "/content/SIDD/input"
    CLEAN_DIR = "/content/SIDD/target"
    SAVE_PATH = "/content/checkpoints_pruned"
    os.makedirs(SAVE_PATH, exist_ok=True)

    EPOCHS = 40               # 40 epochs of training before pruning
    FINETUNE_EPOCHS = 8       # epochs to fine-tune after pruning
    BATCH_SIZE = 8
    LR = 2e-4
    BASE_CH = 32              # smaller base channels for edge
    SEPARABLE = False         # set True to use depthwise-separable convs in conv_block
    PRUNE_AMOUNT = 0.5        # fraction of weights to prune globally
    BEST_PSNR = 0.0

    # ---- Initialize wandb ----
    wandb.init(
        project="sdp_project2",
        config={
            "epochs": EPOCHS,
            "finetune_epochs": FINETUNE_EPOCHS,
            "batch_size": BATCH_SIZE,
            "learning_rate": LR,
            "base_channels": BASE_CH,
            "separable": SEPARABLE,
            "prune_amount": PRUNE_AMOUNT,
            "device": DEVICE,
            "model": "MultiStageRestorationNet",
            "dataset": "SIDD"
        },
        name="40_epochs_denoise_prune"
    )

    # ---- data ----
    transform = T.Compose([T.Resize((256, 256)), T.ToTensor()])
    dataset = SIDD_DenoiseDataset(noisy_dir=NOISY_DIR, clean_dir=CLEAN_DIR, transform=transform)
    if len(dataset) == 0:
        raise RuntimeError("Dataset appears empty; check paths.")

    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # ---- model / optimizer ----
    model = MultiStageRestorationNet(in_ch=3, base_ch=BASE_CH, separable=SEPARABLE).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

    # Log model architecture to wandb
    wandb.watch(model, log="all", log_freq=100)

    # ---- Checkpoint loading with resilience ----
    checkpoint_final = os.path.join(SAVE_PATH, "final_preprune.pth")
    start_epoch = 0

    if os.path.exists(checkpoint_final):
        print("Found pre-prune checkpoint, loading...")
        try:
            checkpoint = torch.load(checkpoint_final, map_location=DEVICE)

            # Handle different checkpoint formats
            if isinstance(checkpoint, dict):
                # New format with state dicts
                if 'model_state_dict' in checkpoint:
                    model.load_state_dict(checkpoint['model_state_dict'])
                    opt.load_state_dict(checkpoint['optimizer_state_dict'])
                    start_epoch = checkpoint.get('epoch', 0) + 1
                    BEST_PSNR = checkpoint.get('best_psnr', 0.0)
                    print(f"Resumed training from epoch {start_epoch}")
                else:
                    # Old format - direct model state dict
                    model.load_state_dict(checkpoint)
                    print("Loaded model weights (old format)")
            else:
                # Very old format - direct model state dict
                model.load_state_dict(checkpoint)
                print("Loaded model weights (very old format)")

        except Exception as e:
            print(f"Error loading checkpoint: {e}")
            print("Starting training from scratch...")
            start_epoch = 0
    else:
        print("No checkpoint found, starting training from scratch...")

    # ---- Train for 40 epochs before pruning ----
    print(f"Starting training for {EPOCHS} epochs to get a well-trained model before pruning...")

    for epoch in range(start_epoch, EPOCHS):
        train_loss, train_psnr, train_l1, train_ssim, train_mse = train_one_epoch(model, train_loader, opt, DEVICE, epoch)
        val_loss, val_psnr, val_l1, val_ssim, val_mse = validate(model, val_loader, DEVICE, epoch)

        print(f"[PrePrune] Epoch {epoch+1}/{EPOCHS} | "
              f"Train Loss: {train_loss:.4f} PSNR: {train_psnr:.4f} | "
              f"Val Loss: {val_loss:.4f} PSNR: {val_psnr:.4f}")

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_psnr": train_psnr,
            "train_l1": train_l1,
            "train_ssim": train_ssim,
            "train_mse": train_mse,
            "val_loss": val_loss,
            "val_psnr": val_psnr,
            "val_l1": val_l1,
            "val_ssim": val_ssim,
            "val_mse": val_mse,
        })

        # Save best model during pre-pruning training
        if val_psnr > BEST_PSNR:
            BEST_PSNR = val_psnr
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
                'best_psnr': BEST_PSNR,
                'loss': val_loss,
            }, os.path.join(SAVE_PATH, "best_preprune_model.pth"))
            print(f"Saved best pre-prune model with PSNR {BEST_PSNR:.4f}")

        # Save checkpoint every 10 epochs
        if (epoch + 1) % 10 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
                'best_psnr': BEST_PSNR,
                'loss': val_loss,
            }, os.path.join(SAVE_PATH, f"checkpoint_epoch_{epoch+1}.pth"))

    # Save final pre-prune model
    torch.save({
        'epoch': EPOCHS,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict(),
        'best_psnr': BEST_PSNR,
        'loss': val_loss,
    }, checkpoint_final)
    print(f"Completed {EPOCHS} epochs of pre-pruning training. Final PSNR: {BEST_PSNR:.4f}")

    # ---- Apply pruning ----
    print("Applying pruning to model...")
    apply_global_unstructured_pruning(model, amount=PRUNE_AMOUNT)

    # Calculate and log sparsity
    total_zeros = 0
    total_params = 0
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            w = module.weight.detach()
            total_zeros += torch.sum(w == 0).item()
            total_params += w.numel()

    sparsity_after_mask = total_zeros / total_params
    print(f"Global sparsity after mask applied: {total_zeros}/{total_params} = {sparsity_after_mask:.2%}")

    # Log sparsity to wandb
    wandb.log({
        "pruning_sparsity_after_mask": sparsity_after_mask,
        "pruning_total_zeros": total_zeros,
        "pruning_total_params": total_params
    })

    # Make pruning permanent (removes hooks and masks)
    remove_pruning_reparametrization(model)

    # Confirm zeros after removal
    total_zeros = 0
    total_params = 0
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            w = module.weight.detach()
            total_zeros += torch.sum(w == 0).item()
            total_params += w.numel()

    final_sparsity = total_zeros / total_params
    print(f"Global sparsity after remove(): {total_zeros}/{total_params} = {final_sparsity:.2%}")

    # Log final sparsity
    wandb.log({
        "pruning_final_sparsity": final_sparsity
    })

    # ---- Fine-tune the pruned model ----
    print("Fine-tuning pruned model...")
    # Lower LR for fine-tune and reset optimizer
    opt = torch.optim.AdamW(model.parameters(), lr=LR*0.5, weight_decay=1e-6)
    finetune_best_psnr = 0.0

    for epoch in range(FINETUNE_EPOCHS):
        train_loss, train_psnr, train_l1, train_ssim, train_mse = train_one_epoch(model, train_loader, opt, DEVICE, epoch)
        val_loss, val_psnr, val_l1, val_ssim, val_mse = validate(model, val_loader, DEVICE, epoch)

        print(f"[FineTune] Epoch {epoch+1}/{FINETUNE_EPOCHS} | "
              f"Train Loss: {train_loss:.4f} PSNR: {train_psnr:.4f} | "
              f"Val Loss: {val_loss:.4f} PSNR: {val_psnr:.4f}")

        # Log fine-tuning metrics to wandb
        wandb.log({
            "finetune_epoch": epoch + 1,
            "finetune_train_loss": train_loss,
            "finetune_train_psnr": train_psnr,
            "finetune_train_l1": train_l1,
            "finetune_train_ssim": train_ssim,
            "finetune_train_mse": train_mse,
            "finetune_val_loss": val_loss,
            "finetune_val_psnr": val_psnr,
            "finetune_val_l1": val_l1,
            "finetune_val_ssim": val_ssim,
            "finetune_val_mse": val_mse,
        })

        # save best fine-tuned model
        if val_psnr > finetune_best_psnr:
            finetune_best_psnr = val_psnr
            torch.save(model.state_dict(), os.path.join(SAVE_PATH, "best_pruned_model.pth"))
            print(f"Saved best pruned model with PSNR {finetune_best_psnr:.4f}")

            # Log best PSNR
            wandb.log({"best_finetune_psnr": finetune_best_psnr})

    # Final save
    torch.save(model.state_dict(), os.path.join(SAVE_PATH, "final_pruned_model.pth"))
    print("Saved final pruned model.")

    # ---- Export to TorchScript for edge deployment ----
    print("Exporting to TorchScript (script mode)...")
    model.eval()
    example = torch.randn(1, 3, 256, 256).to(DEVICE)
    try:
        scripted = torch.jit.trace(model, example)   # tracing is usually fine for deterministic nets
        ts_path = os.path.join(SAVE_PATH, "model_pruned_scripted.pt")
        scripted.save(ts_path)
        print(f"Exported TorchScript to {ts_path}")

        # Log TorchScript export success
        wandb.log({"torchscript_export": "success"})
    except Exception as e:
        print("TorchScript tracing failed:", e)
        try:
            scripted = torch.jit.script(model)
            ts_path = os.path.join(SAVE_PATH, "model_pruned_scripted.pt")
            scripted.save(ts_path)
            print(f"Exported TorchScript (script) to {ts_path}")
            wandb.log({"torchscript_export": "success_script"})
        except Exception as e2:
            print("TorchScript script failed:", e2)
            print("Skipping TorchScript export.")
            wandb.log({"torchscript_export": "failed"})

    # Finish wandb run
    wandb.finish()
    print("All done. Training completed for 40 epochs + fine-tuning.")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Found pre-prune checkpoint, loading...
Loaded model weights (old format)
Starting training for 40 epochs to get a well-trained model before pruning...


Val: 100%|██████████| 2/2 [00:12<00:00,  6.43s/it]


[PrePrune] Epoch 1/40 | Train Loss: 0.2262 PSNR: 14.0957 | Val Loss: 0.1964 PSNR: 15.0368
Saved best pre-prune model with PSNR 15.0368


Val: 100%|██████████| 2/2 [00:12<00:00,  6.16s/it]


...

[FineTune] Epoch 7/8 | Train Loss: 0.0266 PSNR: 31.0484 | Val Loss: 0.0290 PSNR: 29.9026
Saved best pruned model with PSNR 29.9026


Val: 100%|██████████| 2/2 [00:12<00:00,  6.43s/it]


[FineTune] Epoch 8/8 | Train Loss: 0.0268 PSNR: 31.0438 | Val Loss: 0.0289 PSNR: 29.9524
Saved best pruned model with PSNR 29.9524
Saved final pruned model.
Exporting to TorchScript (script mode)...
Exported TorchScript to /content/checkpoints_pruned/model_pruned_scripted.pt


best_finetune_psnr,▁▄▅▆▆▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
finetune_epoch,▁▂▃▄▅▆▇█
finetune_train_l1,█▂▂▁▁▁▁▂
finetune_train_loss,█▃▂▂▂▁▁▁
finetune_train_mse,█▃▂▂▁▁▁▁
finetune_train_psnr,▁▆▅▇▇█▇▇
finetune_train_ssim,█▅▄▃▂▂▁▁
finetune_val_l1,█▄▄▂▂▅▂▁
finetune_val_loss,█▄▄▂▃▄▁▁
+17,...


All done. Training completed for 40 epochs + fine-tuning.


In [10]:
!zip -r denoise_model.zip /content/checkpoints_pruned


  adding: content/checkpoints_pruned/ (stored 0%)
  adding: content/checkpoints_pruned/best_pruned_model.pth (deflated 16%)
  adding: content/checkpoints_pruned/checkpoint_epoch_30.pth (deflated 14%)
  adding: content/checkpoints_pruned/final_preprune.pth (deflated 14%)
  adding: content/checkpoints_pruned/model_pruned_scripted.pt (deflated 17%)
  adding: content/checkpoints_pruned/checkpoint_epoch_10.pth (deflated 17%)
  adding: content/checkpoints_pruned/checkpoint_epoch_20.pth (deflated 15%)
  adding: content/checkpoints_pruned/final_pruned_model.pth (deflated 16%)
  adding: content/checkpoints_pruned/best_preprune_model.pth (deflated 14%)
  adding: content/checkpoints_pruned/checkpoint_epoch_40.pth (deflated 14%)


In [11]:
from google.colab import files
files.download("denoise_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>